In [1]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

In [2]:
train_df = pd.read_csv("imat2009_train_new.txt", header = None)
test_df = pd.read_csv("imat2009_test_new.txt", header = None)

In [3]:
X_train = np.zeros((train_df.shape[0], 245))
y_train = np.zeros((train_df.shape[0]))
queries_train = np.zeros((train_df.shape[0]))
queries_train = queries_train.astype(np.int64)

X_test = np.zeros((test_df.shape[0], 245))
y_test = np.zeros((test_df.shape[0]))
queries_test = np.zeros((test_df.shape[0]))
queries_test = queries_test.astype(np.int64)

In [4]:
for i in range(0, train_df.shape[0]):
    splitted = train_df[0][i].split()
    for j in range(0, len(splitted) - 1):
        if j == 0:
            y_train[i] = float(splitted[j])
            y_train[i] = y_train[i].astype('float64')
        elif splitted[j] == '#':
            queries_train[i] = int(splitted[j + 1])
            break
        else:
            temp = ''
            for x in splitted[j]:
                if x == ':':
                    feat_ind = int(temp)
                    temp = ''
                else:
                    temp += str(x)
            X_train[i][feat_ind - 1] = float(temp)

In [5]:
for i in range(0, test_df.shape[0]):
    splitted = test_df[0][i].split()
    for j in range(0, len(splitted) - 1):
        if j == 0:
            y_test[i] = float(splitted[j])
            y_test[i] = y_test[i].astype('float64')
        elif splitted[j] == '#':
            queries_test[i] = int(splitted[j + 1])
            break
        else:
            temp = ''
            for x in splitted[j]:
                if x == ':':
                    feat_ind = int(temp)
                    temp = ''
                else:
                    temp += str(x)
            X_test[i][feat_ind - 1] = float(temp)

6.27 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
num_documents = X_train.shape[0]
print(num_documents)

77714


In [11]:
X_train.shape[1]

CPU times: total: 0 ns
Wall time: 0 ns


245

In [13]:
from collections import Counter
Counter(y_train).items()

dict_items([(1.0, 20086), (0.0, 25776), (2.0, 24424), (4.0, 952), (3.0, 1744), (0.5, 1982), (1.5, 1033), (0.25, 77), (1.33333, 110), (1.2, 3), (2.37037, 39), (0.666671, 340), (2.33333, 79), (0.333329, 268), (2.16049, 19), (2.5, 337), (2.87037, 26), (1.66667, 107), (2.12037, 4), (2.25, 19), (2.24074, 25), (0.2, 10), (1.6, 6), (0.8, 5), (0.6, 10), (0.875, 1), (2.66667, 31), (3.1625, 2), (1.75, 12), (0.75, 55), (2.61111, 4), (0.222229, 1), (0.4, 5), (1.25, 23), (1.97143, 2), (3.5, 16), (2.24691, 10), (2.16667, 1), (1.95239, 1), (1.4, 4), (3.66667, 5), (3.8, 2), (0.125, 1), (2.05556, 2), (3.33333, 4), (2.2, 5), (2.58025, 2), (1.16667, 2), (2.91358, 1), (2.07407, 3), (2.11729, 1), (3.25, 1), (2.375, 1), (3.21666, 1), (2.74074, 5), (2.12346, 3), (0.166671, 8), (0.833329, 5), (1.14286, 1), (3.53, 1), (3.4, 1), (2.75, 1), (3.58125, 1), (2.40741, 1), (0.583329, 1), (1.8, 1), (2.42857, 1), (2.0463, 1), (1.77143, 1), (3.75, 1), (0.888886, 1)])

In [14]:
max_relevance = np.max(y_train)
print(max_relevance)
y_train /= max_relevance
y_test /= max_relevance

4.0


In [15]:
num_queries = np.unique(queries_train).shape[0]
print(num_queries)

7300


In [16]:
a = np.expand_dims(queries_train, axis = 1)

X_train = np.append(X_train, a, axis = 1)

X_train = X_train[X_train[:, 245].argsort()]

b = np.expand_dims(queries_test, axis = 1)

X_test = np.append(X_test, b, axis = 1)

X_test = X_test[X_test[:, 245].argsort()]

X_train = np.delete(X_train, 245, 1)

X_test = np.delete(X_test, 245, 1)

In [17]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [18]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [19]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [20]:
%%time

fit_model('PairLogit')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 20min 2s
Wall time: 8min 26s


In [21]:
%%time

fit_model('PairLogitPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 3h 18min 33s
Wall time: 51min 56s


In [22]:
%%time

fit_model('YetiRank')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 16min 37s
Wall time: 2min 52s


In [23]:
%%time

fit_model('YetiRankPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 1h 44min 22s
Wall time: 15min 58s
